# Capstone Project: 
  ## Flowers Classification using Convolutional Neural Networks

In [142]:
import numpy as np
import pandas as pd
import keras

In [2]:
from keras.layers import Input, Conv2D, MaxPool2D, Dropout, Activation, Dense, Flatten
from keras.models import Sequential
from keras.activations import relu, softmax
from keras.optimizers import Adam

In [3]:
from PIL import Image
import random

In [4]:
import cv2
import os
import urllib.request
from urllib.request import Request, urlopen
from urllib.error import URLError
# import socket  
# socket.setdefaulttimeout(1)

In [50]:
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

## Data Collection

### Scrap images from http://www.image-net.org 


In [ ]:
category={'Sunflower':'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n11978961',
          'Peony':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11719286',
          'Nigella':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11736851',
          'Spathiphyllum':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11792341',
          'Ragged_robin':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11811706',
          'Soapwort':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11814584',
          'Ice_plant':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11821184',
          'Spring_beauty':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11859472',
          'African_daisy':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11925303',
          'Cornflower':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11947802'
          }

In [ ]:
# import cv2
# import os
# import urllib.request
# from urllib.request import Request, urlopen
# from urllib.error import URLError
# import socket  
# socket.setdefaulttimeout(1)

# with urllib.request.urlopen('http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n11978961') as sunflower:
#     sunflower_html = sunflower.read().decode()
    


def get_urls(urls_links):
    url_list=urllib.request.urlopen(urls_links).read().decode().split('\r\n')
    return url_list

In [144]:
def download_images(urls_link,category_name):
    if not os.path.exists(category_name):
        os.makedirs(category_name)
    count=1
    url_list=get_urls(urls_link)
    for url in url_list:
        try:
            path_name=str(category_name)+'/'+str(count)+'.'+str(category_name)+'.jpg'
            urllib.request.urlretrieve(url,path_name)
             
            img=cv2.imread(path_name)
            resized_image=cv2.resize(img,(100,100))
            cv2.imwrite(path_name,resized_image)
        except Exception as e:
            print(str(count)+str(e))
        count+=1

In [ ]:
for cate in category:
    download_images(category['cate'],cate)


## Data Preprocessing 

In [66]:
all_image_dir='/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image'

In [6]:
def label_img(img):
    img_label = img.split('.')[-2]
    return img_label

In [7]:
def jpg_image_to_array(path):
    """
  Loads JPEG image into 3D Numpy array of shape 
  (width, height, channels)
  """
    img=Image.open(path)
    img_arr=np.asarray(img).reshape((img.size[1], img.size[0],3))
    return img_arr
    

In [ ]:
# def create_image_dataset(image_dir):
#     img_dataset=[]
#     for img in os.listdir(image_dir):
#         try: 
#             img_peth=str(image_dir)+'/'+str(img)
#             image_array=jpg_image_to_array(img_peth)
#             label=label_img(img)
#             img_dataset.append([image_array,label])  
#         except Exception as e:
#             print(str(e))
# #     np.save('image_dataset.npy',img_dataset)
# #     return img_dataset
#     return img_dataset

In [ ]:
# image_data_test2=create_image_dataset(all_image_dir)
# image_data_test2

In [11]:
def create_image_list(image_dir):
    img_dataset=[]
    label_dataset=[]
    for img in os.listdir(image_dir):
        try: 
            img_path=str(image_dir)+'/'+str(img)
            image_array=jpg_image_to_array(img_path)
            label=label_img(img)
            img_dataset.append([image_array])  
#             label_dataset.append([label])
        except Exception as e:
            print(str(e))
#     np.save('image_dataset.npy',img_dataset)
#     return img_dataset
    return img_dataset

In [15]:
image_list=create_image_list(all_image_dir)

cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/.DS_Store'
cannot reshape array of size 16320 into shape (120,136,3)
cannot reshape array of size 12100 into shape (110,110,3)
cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/370.Ragged_robin.jpg'
cannot reshape array of size 7000 into shape (70,100,3)
cannot reshape array of size 9216 into shape (96,96,3)
cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/438.Spring_beauty.jpg'
cannot reshape array of size 90000 into shape (300,300,3)
cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/472.Sunflower.jpg'
cannot reshape array of size 7000 into shape (70,100,3)
cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/725.Spring_beauty.j

In [16]:
image_list_arr=np.array(image_list)
image_list_arr.shape

(8109, 1, 100, 100, 3)

In [17]:
image_list_reshape=image_list_arr.reshape(8109,100,100,3)
image_list_reshape.shape

(8109, 100, 100, 3)

In [18]:
def create_label_list(image_dir):
    img_dataset=[]
    label_dataset=[]
    for img in os.listdir(image_dir):
        try: 
            img_path=str(image_dir)+'/'+str(img)
            image_array=jpg_image_to_array(img_path)
            label=label_img(img)
#             img_dataset.append([image_array])  
            label_dataset.append([label])
        except Exception as e:
            print(str(e))
#     np.save('image_dataset.npy',img_dataset)
#     return img_dataset
    return label_dataset

In [19]:
label_list=create_label_list(all_image_dir)

cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/.DS_Store'
cannot reshape array of size 16320 into shape (120,136,3)
cannot reshape array of size 12100 into shape (110,110,3)
cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/370.Ragged_robin.jpg'
cannot reshape array of size 7000 into shape (70,100,3)
cannot reshape array of size 9216 into shape (96,96,3)
cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/438.Spring_beauty.jpg'
cannot reshape array of size 90000 into shape (300,300,3)
cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/472.Sunflower.jpg'
cannot reshape array of size 7000 into shape (70,100,3)
cannot identify image file '/Users/Dan/Desktop/GA/DSI_Plus/capstone_project/image_download/test/all_flower_image/725.Spring_beauty.j

In [20]:
label_list_arr=np.array(label_list)
label_list_arr.shape

(8109, 1)

In [28]:
import random

a = image_list_arr
b = label_list_arr

c = list(zip(a, b))

random.shuffle(c)

a1, b1 = zip(*c)


In [29]:
a2=np.array(a1)
b2=np.array(b1)

In [30]:
a2=a2.reshape(8109, 100, 100, 3)
print(a2.shape,b2.shape)

(8109, 100, 100, 3) (8109, 1)


In [31]:
8109*0.8

6487.200000000001

In [32]:
X_train=a2[:6487]
X_test=a2[6487:]
y_train=b2[:6487]
y_test=b2[6487:]

In [34]:
X_train_rescale=X_train/255
X_test_rescale=X_test/255

In [35]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_train_1hot = encoder.fit_transform(y_train)
y_test_1hot=encoder.fit_transform(y_test)


In [37]:
y_train_1hot

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [38]:
y_test_1hot

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [ ]:
# y_train_test11 = keras.utils.to_categorical(y_train_test1, 10)
# y_test_test11 = keras.utils.to_categorical(y_test_test1, 10)

## Model Building

In [57]:
cnn1 = Sequential([
    Conv2D(50, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
    MaxPool2D((2,2)),
    
    Conv2D(100, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
    Dropout(.5),
    MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(150, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [58]:
cnn1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 98, 98, 50)        1400      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 49, 49, 50)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 47, 47, 100)       45100     
_________________________________________________________________
dropout_7 (Dropout)          (None, 47, 47, 100)       0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 52900)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               6771328   
__________

In [59]:
cnn1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])

In [60]:
cnn1.fit(X_train_rescale, y_train_1hot, validation_split=.2, epochs=5)

Train on 5189 samples, validate on 1298 samples
Epoch 1/5
5189/5189 [==============================] - 176s - loss: 2.0496 - acc: 0.3103 - val_loss: 2.0624 - val_acc: 0.3521
Epoch 2/5
5189/5189 [==============================] - 153s - loss: 1.6652 - acc: 0.4282 - val_loss: 1.9310 - val_acc: 0.2935
Epoch 3/5
5189/5189 [==============================] - 156s - loss: 1.4855 - acc: 0.4962 - val_loss: 1.6773 - val_acc: 0.4584
Epoch 4/5
5189/5189 [==============================] - 163s - loss: 1.3337 - acc: 0.5614 - val_loss: 1.5686 - val_acc: 0.4761
Epoch 5/5
5189/5189 [==============================] - 164s - loss: 1.1953 - acc: 0.6101 - val_loss: 1.5297 - val_acc: 0.4669


In [61]:
loss, accuracy1 = cnn1.evaluate(X_test_rescale, y_test_1hot)

1622/1622 [==============================] - 14s    


In [62]:
accuracy1

0.45314426648484502

In [ ]:
'''
number_of_conv   dropout     num_fillter    Dense   LR   epochs   train_acc   val_acc   test_acc
    2               0.25       50 100        128   0.5     5       0.7          0.61       0.57
    3               0.25       50 100 200    128   0.5     5       0.67         0.60       0.57 
    2               0.5        50 100         64   0.2     5       0.49         0.53       0.58
    2               0.5        50 100        128   0.2    20       0.85         0.63       0.63  
    
    
    
'''

## Data Augmentation

In [51]:
cnn2 = Sequential([
    Conv2D(50, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
    MaxPool2D((2,2)),
    
    Conv2D(100, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(150, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [52]:
cnn2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 98, 98, 50)        1400      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 50)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 47, 47, 100)       45100     
_________________________________________________________________
batch_normalization_5 (Batch (None, 47, 47, 100)       400       
_________________________________________________________________
dropout_5 (Dropout)          (None, 47, 47, 100)       0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 52900)             0         
__________

In [ ]:
# from keras import optimizers
# sgd = optimizers.SGD(lr=.2, momentum=0.9, nesterov=True)

In [53]:
cnn2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.5),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])


In [54]:
# X_train_test1=a2[:6533]
# X_test_test1=a2[6533:]
# y_train_test1=b2[:6533]
# y_test_test1=b2[6533:]

imgen_train = ImageDataGenerator(
    rotation_range=8,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
)

imgen_test=ImageDataGenerator()



In [55]:
train_generator = imgen_train.flow(X_train_rescale, y_train_1hot, batch_size=64)
test_generator = imgen_test.flow(X_test_rescale, y_test_1hot, batch_size=64)


In [56]:
cnn2.fit_generator(train_generator, steps_per_epoch=X_train_rescale.shape[0] // 64, epochs=5, 
                    validation_data=test_generator, validation_steps=len(y_test_1hot)//64)

Epoch 1/5
101/101 [==============================] - 227s - loss: 2.1767 - acc: 0.3244 - val_loss: 2.3420 - val_acc: 0.1388
Epoch 2/5
101/101 [==============================] - 233s - loss: 1.7327 - acc: 0.4368 - val_loss: 2.3746 - val_acc: 0.1791
Epoch 3/5
101/101 [==============================] - 238s - loss: 1.5952 - acc: 0.4817 - val_loss: 2.2884 - val_acc: 0.2330
Epoch 4/5
101/101 [==============================] - 229s - loss: 1.4969 - acc: 0.5122 - val_loss: 2.2017 - val_acc: 0.3338
Epoch 5/5
101/101 [==============================] - 230s - loss: 1.4118 - acc: 0.5361 - val_loss: 2.1200 - val_acc: 0.3498


## Add Data

In [103]:
new_image_dir='/Users/Dan/Desktop/new_image/all_flower_resize_new2'

In [104]:
image_list2=create_image_list(new_image_dir)

cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/.DS_Store'
cannot reshape array of size 16320 into shape (120,136,3)
cannot reshape array of size 12100 into shape (110,110,3)
cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/370.Ragged_robin.jpg'
cannot reshape array of size 7000 into shape (70,100,3)
cannot reshape array of size 9216 into shape (96,96,3)
cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/438.Spring_beauty.jpg'
cannot reshape array of size 90000 into shape (300,300,3)
cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/472.Sunflower.jpg'
cannot reshape array of size 7000 into shape (70,100,3)
cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/725.Spring_beauty.jpg'
cannot reshape array of size 2700 into shape (60,45,3)
cannot reshape array of size 36936 into shape (216,171,3)
cannot reshape array of size 13440 into shape (1

In [ ]:
def create_image_list(image_dir):
    img_dataset=[]
    label_dataset=[]
    for img in os.listdir(image_dir):
        try: 
            img_peth=str(image_dir)+'/'+str(img)
            image_array=jpg_image_to_array(img_peth)
            label=label_img(img)
            img_dataset.append([image_array])  
#             label_dataset.append([label])
        except Exception as e:
            print(str(e))
#     np.save('image_dataset.npy',img_dataset)
#     return img_dataset
    return img_dataset

In [105]:
image_list2_arr=np.array(image_list2)
image_list2_arr.shape

(11755, 1, 100, 100, 3)

In [106]:
image_list2_reshape=image_list2_arr.reshape(11755,100,100,3)
image_list2_reshape.shape

(11755, 100, 100, 3)

In [107]:
label_list2=create_label_list(new_image_dir)

cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/.DS_Store'
cannot reshape array of size 16320 into shape (120,136,3)
cannot reshape array of size 12100 into shape (110,110,3)
cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/370.Ragged_robin.jpg'
cannot reshape array of size 7000 into shape (70,100,3)
cannot reshape array of size 9216 into shape (96,96,3)
cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/438.Spring_beauty.jpg'
cannot reshape array of size 90000 into shape (300,300,3)
cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/472.Sunflower.jpg'
cannot reshape array of size 7000 into shape (70,100,3)
cannot identify image file '/Users/Dan/Desktop/new_image/all_flower_resize_new2/725.Spring_beauty.jpg'
cannot reshape array of size 2700 into shape (60,45,3)
cannot reshape array of size 36936 into shape (216,171,3)
cannot reshape array of size 13440 into shape (1

In [ ]:
cnn2.fit(X_train_test1_rescale, y_train_test1_1hot, validation_split=.2, epochs=5)

In [ ]:
loss, accuracy = cnn2.evaluate(X_test_test1_rescale, y_test_test1_1hot)

In [ ]:
accuracy

In [ ]:
def create_label_list(image_dir):
    img_dataset=[]
    label_dataset=[]
    for img in os.listdir(image_dir):
        try: 
            img_peth=str(image_dir)+'/'+str(img)
            image_array=jpg_image_to_array(img_peth)
            label=label_img(img)
#             img_dataset.append([image_array])  
            label_dataset.append([label])
        except Exception as e:
            print(str(e))
#     np.save('image_dataset.npy',img_dataset)
#     return img_dataset
    return label_dataset

In [108]:
label_list2_arr=np.array(label_list2)
label_list2_arr.shape

(11755, 1)

In [109]:
import random

aa = image_list2_arr
bb = label_list2_arr

cc = list(zip(aa, bb))

random.shuffle(cc)

aa1, bb1 = zip(*cc)



In [ ]:
# f2(label_list2)

In [110]:
aa2=np.array(aa1)
bb2=np.array(bb1)

In [112]:
aa2=aa2.reshape(11755, 100, 100, 3)
print(aa2.shape,bb2.shape)

(11755, 100, 100, 3) (11755, 1)


In [113]:
11758*0.8


9406.4

In [114]:
X_train2=aa2[:9406]
X_test2=aa2[9406:]
y_train2=bb2[:9406]
y_test2=bb2[9406:]

In [115]:
X_train2_rescale=X_train2/255
X_test2_rescale=X_test2/255

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# X_train_test1=a2[:6533]
# X_test_test1=a2[6533:]
# y_train_test1=b2[:6533]
# y_test_test1=b2[6533:]

imgen_train = ImageDataGenerator(
    rotation_range=8,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='constant'
)

imgen_test=ImageDataGenerator()


In [ ]:
train_generator = imgen_train.flow(X_train_test1_rescale, y_train_test1_1hot, batch_size=64)
test_generator = imgen_test.flow(X_test_test1_rescale, y_test_test1_1hot, batch_size=64)



In [ ]:
cnn2.fit_generator(train_generator, steps_per_epoch=X_train_test1_rescale.shape[0] // 64, epochs=5, 
                    validation_data=test_generator, validation_steps=len(y_test_test1_1hot)//64)

In [116]:
y_train2_1hot = encoder.fit_transform(y_train2)
y_test2_1hot = encoder.fit_transform(y_test2)


In [127]:
cnn3 = Sequential([
    Conv2D(50, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
    MaxPool2D((2,2)),
    
    Conv2D(100, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
    Dropout(.5),
    MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(150, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
#     BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [128]:
cnn3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 98, 98, 50)        1400      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 49, 49, 50)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 47, 47, 100)       45100     
_________________________________________________________________
dropout_15 (Dropout)         (None, 47, 47, 100)       0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 23, 23, 100)       0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 52900)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               6771328   
__________

In [130]:
cnn3.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.2),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])

In [131]:
cnn3.fit(X_train2_rescale, y_train2_1hot, validation_split=.2, epochs=20)

Train on 7524 samples, validate on 1882 samples
Epoch 1/20
7524/7524 [==============================] - 294s - loss: 1.9844 - acc: 0.2937 - val_loss: 2.0600 - val_acc: 0.2147
Epoch 2/20
7524/7524 [==============================] - 210s - loss: 1.6213 - acc: 0.4520 - val_loss: 1.6241 - val_acc: 0.4915
Epoch 3/20
7524/7524 [==============================] - 209s - loss: 1.4643 - acc: 0.5109 - val_loss: 1.6141 - val_acc: 0.4192
Epoch 4/20
7524/7524 [==============================] - 211s - loss: 1.3478 - acc: 0.5561 - val_loss: 1.4885 - val_acc: 0.5484
Epoch 5/20
7524/7524 [==============================] - 207s - loss: 1.2661 - acc: 0.5845 - val_loss: 1.4841 - val_acc: 0.4936
Epoch 6/20
7524/7524 [==============================] - 202s - loss: 1.2012 - acc: 0.6066 - val_loss: 1.3532 - val_acc: 0.5893
Epoch 7/20
7524/7524 [==============================] - 201s - loss: 1.1521 - acc: 0.6227 - val_loss: 1.3484 - val_acc: 0.5956
Epoch 8/20
7524/7524 [==============================] - 202s - 

In [140]:
loss, accuracy3 = cnn3.evaluate(X_test2_rescale, y_test2_1hot)

2349/2349 [==============================] - 26s    


In [141]:
accuracy3

0.69306087704504638

In [ ]:
## Data Augmentation

In [145]:
cnn4 = Sequential([
    Conv2D(50, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
    MaxPool2D((2,2)),
    
    Conv2D(100, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(150, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [146]:
cnn4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 98, 98, 50)        1400      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 49, 49, 50)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 47, 47, 100)       45100     
_________________________________________________________________
batch_normalization_10 (Batc (None, 47, 47, 100)       400       
_________________________________________________________________
dropout_17 (Dropout)         (None, 47, 47, 100)       0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 23, 23, 100)       0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 52900)             0         
__________

In [147]:
cnn4.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.2),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])

In [148]:
imgen_train2 = ImageDataGenerator(
    rotation_range=8,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
)

imgen_test2=ImageDataGenerator()




In [149]:
train_generator2 = imgen_train2.flow(X_train2_rescale, y_train2_1hot, batch_size=64)
test_generator2 = imgen_test2.flow(X_test2_rescale, y_test2_1hot, batch_size=64)



In [151]:
cnn4.fit_generator(train_generator2, steps_per_epoch=X_train2_rescale.shape[0] // 64, epochs=10, 
                    validation_data=test_generator2, validation_steps=len(y_test2_1hot)//64)

Epoch 1/10The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.

146/146 [==============================] - 312s - loss: 1.3354 - acc: 0.5674 - val_loss: 1.4806 - val_acc: 0.5794
Epoch 2/10
146/146 [==============================] - 311s - loss: 1.3088 - acc: 0.5684 - val_loss: 1.6450 - val_acc: 0.5632
Epoch 3/10
146/146 [==============================] - 307s - loss: 1.2945 - acc: 0.5819 - val_loss: 1.5618 - val_acc: 0.5786
Epoch 4/10
146/146 [==============================] - 306s - loss: 1.2770 - acc: 0.5827 - val_loss: 1.5941 - val_acc: 0.5733
Epoch 5/10
146/146 [==============================] - 304s - loss: 1.2729 - acc: 0.5809 - val_loss: 1.6811 - val_acc: 0.5851
Epoch 6/10
146/146 [==============================] - 306s - loss: 1.2605 - acc: 0.5888 - val_loss: 1.5607 - val_acc: 0.5812
Epoch 7/10
146/146 [==============================] - 307s - loss: 1.2396 - acc: 0.5980 - val_loss: 1.3